In [1]:
!pip install openai pyPdf tiktoken
!pip install langchain langchain-community langchain-core

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.4/360.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 999.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.6/990.6 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.0/384.0 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.2/140.2 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2

In [2]:
# 언어 모델 만들기
import os

os.environ["OPENAI_API_KEY"] = '45462e152b294e11bbcd89f632b6a4df'
os.environ["OPENAI_API_TYPE"] = 'azure'
os.environ["OPENAI_API_VERSION"] = '2023-05-15'
os.environ["AZURE_OPENAI_ENDPOINT"] = 'https://sktflyailab.openai.azure.com/'


In [3]:
from langchain.embeddings import AzureOpenAIEmbeddings

In [4]:
embedding_model = AzureOpenAIEmbeddings(
    deployment='dev-text-embedding-ada-002',
    chunk_size =500
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `AzureOpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import AzureOpenAIEmbeddings`.
  warn_deprecated(


In [5]:
embedding_examples = {
    '안녕하세요',
    '제 이름은 홍길동 입니다.',
    '이름은 무엇 입니까?',
    '랭체인은 유용합니다.',
    'Hello World!'
}

In [7]:
embeddings = embedding_model.embed_documents(embedding_examples)

In [8]:
print('Total embeddings'+ str(len(embeddings)))
print('Dimension of each embedding'+str(len(embeddings[0])))

Total embeddings5
Dimension of each embedding1536


In [10]:
embeddings[0] # 1536개의 좌표계

[0.0023968015664583057,
 0.00037464759016735024,
 -0.0021421905636645716,
 -0.02572514697622958,
 -0.011554940797464119,
 0.000990939961826284,
 -0.014635420401543567,
 0.003445107502759677,
 9.98994737429367e-05,
 -0.027736887428024812,
 0.02377627079420838,
 0.0050042072842234955,
 -0.02763630087109631,
 -0.010190728401371791,
 0.007789212247511062,
 0.011680674924947263,
 0.024995889781885352,
 -0.014145058608210822,
 0.007342856840003909,
 0.0096626461835296,
 -0.007078815731082814,
 0.008518468976194035,
 0.010310176101877533,
 0.005815190823241506,
 -0.00611695189101079,
 0.0019190129763263094,
 0.004884760049379011,
 -0.018860078144281096,
 0.03756927459052441,
 -0.02411575275214837,
 0.016043639649122744,
 -0.012240190208998983,
 -0.00313234472211868,
 -0.024505527988552607,
 0.009769520098758021,
 -0.011831555691662535,
 0.0027504279850974504,
 -0.012485372036987871,
 0.015301809600823708,
 -0.018558316610850552,
 0.00866934881158679,
 0.00981352695024487,
 0.00488790372852897

In [11]:
q = '이 대화에서 언급된 이름은 무엇 입니까?'
a = '이 대화에서 언급된 이름은 홍길동 입니다.'

embedding_q =embedding_model.embed_query(q)
embedding_a =embedding_model.embed_query(a)

In [12]:
from numpy import dot
from numpy.linalg import norm
import numpy as np

def cosine_similarity(a, b):
    return dot(a, b)/(norm(a)*norm(b)) # 코사인 세타 => 두 벡터의 내적 / 각각 백터의 길이 의 곱


In [15]:
print(cosine_similarity(embedding_q, embedding_a))
print(cosine_similarity(embedding_q, embeddings[1]))
print(cosine_similarity(embedding_q, embeddings[3]))


0.9083706037134818
0.8554046304867811
0.773619857314492


In [17]:
# Hugging Face
!pip install sentence_transformers

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [18]:
from langchain.embeddings import HuggingFaceEmbeddings

model_name = 'BAAI/bge-small-en'
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}

hf = HuggingFaceEmbeddings(
    model_name =model_name,
    model_kwargs = model_kwargs,
    encode_kwargs = encode_kwargs
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/sett

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/90.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [19]:
embedding_examples={
    'Today is monday',
    'Weather is nice today',
    'What is the problem',
    'Langchain is useful',
    'Hello World!',
    'My name is moris'
}


In [20]:
embeddings = hf.embed_documents(embedding_examples)

print('Total embeddings'+ str(len(embeddings)))
print('Dimension of each embedding'+str(len(embeddings[0]))) # 데이터를 384차원에 임베딩

Total embeddings6
Dimension of each embedding384


In [24]:
q = 'Hello? who is this?'
a = 'hi this is harrison'

embedding_q =hf.embed_query(q)
embedding_a =hf.embed_query(a)

print(cosine_similarity(embedding_q, embedding_a))
print(cosine_similarity(embedding_q, embeddings[1]))
print(cosine_similarity(embedding_q, embeddings[3])) # 'What is the problem'
print(cosine_similarity(embedding_q, embeddings[5])) # 'My name is moris'

0.8788961229253981
0.7866166306499176
0.8280202752367009
0.791096977761672


In [25]:
embedding_examples = {
    '안녕하세요',
    '제 이름은 홍길동 입니다.',
    '이름은 무엇 입니까?',
    '랭체인은 유용합니다.',
    '홍길동 아버지의 이름은 홍상직 입니다.'
}

embeddings = hf.embed_documents(embedding_examples)

q = '홍길동은 아버지를 아버지라 부르지 못했습니다. 홍길동의 아버지 이름은 무엇 입니까?'
a = '홍길동 아버지는 엄했습니다.'

embedding_q =hf.embed_query(q)
embedding_a =hf.embed_query(a)

print(cosine_similarity(embedding_q, embedding_a))
print(cosine_similarity(embedding_q, embeddings[1])) # '안녕하세요'
print(cosine_similarity(embedding_q, embeddings[3])) # '이름은 무엇 입니까?'
print(cosine_similarity(embedding_q, embeddings[4]))

0.9471720459128122
0.9117469963179475
0.8941495884428019
0.887939435078331


- 한글 데이터를 입력했을 때 영문모델은 변별력이 없다.
  관련없는 글 끼리 유사도가 다 높게 나옴..

- 한글 임베딩 모델 사용 !

In [26]:
model_name = "jhgan/ko-sbert-nli"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}
ko = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.46k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/620 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/538 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [32]:
embedding_examples = {
    '안녕하세요',
    '제 이름은 홍길동 입니다.',
    '이름은 무엇 입니까?',
    '랭체인은 유용합니다.',
    '홍길동 아버지의 이름은 홍상직 입니다.'
}

embeddings = ko.embed_documents(embedding_examples)
print('Total embeddings'+ str(len(embeddings)))
print('Dimension of each embedding'+str(len(embeddings[0]))) # 데이터를 768차원에 임베딩


Total embeddings5
Dimension of each embedding768


In [33]:
q = '홍길동은 아버지를 아버지라 부르지 못했습니다. 홍길동의 아버지 이름은 무엇 입니까?'
a = '홍길동 아버지는 엄했습니다.'

embedding_q = ko.embed_query(q)
embedding_a = ko.embed_query(a)


print(cosine_similarity(embedding_q, embedding_a))
print(cosine_similarity(embedding_q, embeddings[1])) # '안녕하세요'
print(cosine_similarity(embedding_q, embeddings[3])) # '이름은 무엇 입니까?'
print(cosine_similarity(embedding_q, embeddings[4])) # '랭체인은 유용합니다.'

0.4578790878920828
0.17183722736403995
0.02163377903419463
0.5497885531132914
